In [330]:
from random import randbytes

In [331]:
GOST_ENC_INDICES = (0, 1, 2, 3, 4, 5, 6, 7,
                    0, 1, 2, 3, 4, 5, 6, 7,
                    0, 1, 2, 3, 4, 5, 6, 7,
                    7, 6, 5, 4, 3, 2, 1, 0)

GOST_DEC_INDICES = (0, 1, 2, 3, 4, 5, 6, 7,
                    7, 6, 5, 4, 3, 2, 1, 0,
                    7, 6, 5, 4, 3, 2, 1, 0,
                    7, 6, 5, 4, 3, 2, 1, 0)

GOST_SBLOCK = ((9, 6, 3, 2, 8, 11, 1, 7, 10, 4, 14, 15, 12, 0, 13, 5),
             (3, 7, 14, 9, 8, 10, 15, 0, 5, 2, 6, 12, 11, 4, 13, 1),
             (14, 4, 6, 2, 11, 3, 13, 8, 12, 15, 5, 10, 0, 7, 1, 9),
             (14, 7, 10, 12, 13, 1, 3, 9, 0, 2, 11, 4, 15, 8, 5, 6),
             (11, 5, 1, 9, 8, 13, 15, 0, 14, 4, 2, 3, 12, 7, 10, 6),
             (3, 10, 13, 12, 1, 2, 0, 11, 7, 5, 9, 4, 8, 15, 14, 6),
             (1, 13, 2, 9, 7, 10, 6, 0, 8, 12, 4, 5, 15, 3, 11, 14),
             (11, 10, 15, 5, 0, 12, 14, 8, 6, 2, 3, 9, 1, 7, 13, 4))

In [341]:
class GOSTError(Exception):
    pass


class GOST:
    def __init__(self, key: str, iv: str = None, mode: str = "ECB"):
        if len(key) != 64:
            raise GOSTError(f"Key length must be 256 bits (32 bytes)! ({len(key)//2} bytes entered)")
        
        try:
            self.key = int(key, 16)
        except ValueError:
            raise GOSTError("The entered key is not a hexadecimal value!")
            
        
        if iv:
            if len(iv) != 16:
                raise GOSTError(f"IV length must be 64 bits (8 bytes)! ({len(iv)//2} bytes entered)")

            try:
                self.iv = int(iv, 16)
            except:
                raise GOSTError("The entered IV is not a hexadecimal value!")
                
        if iv is None and mode != "ECB":
            raise GOSTError(f"Encryption in '{mode}' mode requires an initialization vector!")
            
        self.subkeys = tuple((self.key >> (32 * i)) & 0xFFFFFFFF for i in range(8))
        
        self._modes = {"ECB": self._ECB,
                       "CBC": self._CBC,
                       "CFB": self._CFB,
                       "OFB": self._OFB}
        
        self._mode = self._modes.get(mode)
    
    def _transform(self, block: int, mode: str = "encrypt"):
        chunk_l = block >> 32
        chunk_r = block & 0xFFFFFFFF
        
        match mode:
            case "encrypt":
                indices = GOST_ENC_INDICES
            
            case "decrypt":
                indices = GOST_DEC_INDICES
            
            case _:
                raise GOSTError(f"Invalid processing mode! -> {mode}")
        
        for i in indices:
            chunk_l, chunk_r = chunk_r ^ self._f(chunk_l, self.subkeys[i]), chunk_l
            
        return (chunk_r << 32) | chunk_l
            
    def _f(self, chunk: int, key: int):
        k = (chunk + key) % 4_294_967_296
        
        new_chunk = 0
        for i in range(8):
            new_chunk |= GOST_SBLOCK[7-i][(k >> (4 * i)) & 0b1111] << (4 * i)
        
        return ((new_chunk << 11) | (new_chunk >> 21)) & 0xFFFFFFFF
    
    def _ECB(self, data: bytes, mode: str = "encrypt"):
        processed_data = bytes()
        
        for pos in range(0, len(data), 8):
            block = int.from_bytes(data[pos:pos+8], "little")
            processed_block = self._transform(block, mode)
            processed_data += processed_block.to_bytes(8, "little")

        return processed_data
    
    def _CBC(self, data: bytes, mode: str = "encrypt"):
        processed_data = bytes()
        vector = self.iv
        
        for pos in range(0, len(data), 8):
            block = int.from_bytes(data[pos:pos+8], "little")
            
            match mode:
                case "encrypt":
                    processed_block = self._transform(block ^ vector, "encrypt")
                    vector = processed_block
                
                case "decrypt":
                    processed_block = self._transform(block, "decrypt") ^ vector
                    vector = block
                    
                case _:
                    raise GOSTError(f"Invalid processing mode! -> {mode}")
                    
            processed_data += processed_block.to_bytes(8, "little")
            
        return processed_data
    
    def _CFB(self, data: bytes, mode: str = "encrypt"):
        processed_data = bytes()
        vector = self.iv
        
        for pos in range(0, len(data), 8):
            block = int.from_bytes(data[pos:pos+8], "little")
            
            match mode:
                case "encrypt":
                    processed_block = self._transform(vector, "encrypt") ^ block
                    vector = processed_block
                
                case "decrypt":
                    processed_block = self._transform(vector, "encrypt") ^ block
                    vector = block
                
                case _:
                    raise GOSTError(f"Invalid processing mode! -> {mode}")
                    
            processed_data += processed_block.to_bytes(8, "little")
            
        return processed_data 
    
    def _OFB(self, data: bytes, mode: str = "encrypt"):
        processed_data = bytes()
        vector = self.iv

        for pos in range(0, len(data), 8):
            block = int.from_bytes(data[pos:pos+8], "little")
            
            match mode:
                case "encrypt":
                    vector = self._transform(vector, "encrypt")
                    processed_block = vector ^ block
                
                case "decrypt":
                    vector = self._transform(vector, "encrypt")
                    processed_block = vector ^ block
                
                case _:
                    raise DESError(f"Invalid processing mode! -> {mode}")
                    
            processed_data += processed_block.to_bytes(8, "little")
            
        return processed_data 
    
    def _data_processing(self, data: bytes or str, mode: str = "encrypt"):               
        match mode, data:
            case "encrypt", str():
                data_bytes = data.encode("utf-8")
                
            case "decrypt", str():
                data_bytes = bytes.fromhex(data)
            
            case _, bytes():
                pass
                
            case _:
                raise GOSTError(f"Invalid processing mode! -> {mode}")
                
        if mode == "encrypt" and (k := len(data_bytes) % 8) != 0:
            data_bytes += b"\00" * (8 - k)
                
        processed_data = self._mode(data_bytes, mode)
        
        match mode, data:
            case "encrypt", str():
                return processed_data.hex()
                
            case "decrypt", str():
                return processed_data.decode("utf-8")
            
            case _, bytes():
                return processed_data
                
            case _:
                raise GOSTError(f"Invalid processing mode! -> {mode}")
    
    def encrypt(self, data: bytes or str):
        return self._data_processing(data, "encrypt")
    
    def decrypt(self, data: bytes or str):
        return self._data_processing(data, "decrypt")
    
    def make(self, data: bytes or str, mode: str = "encrypt"):
        match mode:
            case "encrypt":
                return self.encrypt(data)
            
            case "decrypt":
                return self.decrypt(data)
            
            case _:
                raise DESError(f"Invalid processing mode! -> {mode}")


In [342]:
key = randbytes(32).hex()
print(key)
iv = randbytes(8).hex()
print(iv)

3c5a80c82b2ef0dd3fff015f50350325b5693272b702156213c866b58c39cb3c
43abf5d3028aedf7


In [343]:
gost = GOST(key=key, iv=iv, mode="OFB")
gost.subkeys

(2352597820,
 331900597,
 3070367074,
 3043570290,
 1345651493,
 1073676639,
 724496605,
 1012564168)

In [344]:
m = "hello, World !!@ 🙂 Привет Мир!"

In [345]:
%%time
encrypted = gost.encrypt(m)
print(encrypted)
decrypted = gost.decrypt(encrypted)
print(decrypted)

ccb3f8e77ba33ddaff224ef1a484232bfba6b7b30c89eab95432a6ce5b1099784a79ff07844ad8d7e6f47b1b26869412
hello, World !!@ 🙂 Привет Мир!      
CPU times: user 2.5 ms, sys: 162 µs, total: 2.67 ms
Wall time: 2.6 ms
